In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent import ODEAgent
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import torch.nn as nn
import torch
import gym
from cs285.infrastructure import pytorch_util as ptu
from torchdiffeq import odeint
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax

In [2]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)
device = "gpu" if torch.cuda.is_available() else "cpu"

gpu


In [16]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
agent_key, key = jax.random.split(key)
mb_agent = ODEAgent(
    env=env,
    hidden_size=128,
    num_layers=4,
    ensemble_size=10,
    train_timestep=0.005,
    mpc_horizon_steps=100,
    mpc_timestep=0.005,
    mpc_strategy="cem",
    mpc_discount=0.9,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
    key=agent_key
)
replay_buffer = ReplayBufferTrajectories(seed=42)
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=10, max_length=200)
replay_buffer.add_rollouts(trajs)

100%|██████████| 10/10 [00:00<00:00, 56.67it/s]


In [13]:
for n in trange(2):
    for i in range(mb_agent.ensemble_size):
        traj = replay_buffer.sample_rollout()
        loss = mb_agent.update(i, jnp.array(traj["observations"]), jnp.array(traj["actions"]), jnp.cumsum(traj["dts"]), discount=0.9)
# TODO: change the implementation to allow sampling batched rollouts

100%|██████████| 2/2 [02:42<00:00, 81.15s/it]


In [14]:
loss

0.2549184560775757

In [17]:
batch_size = 64
for n in trange(2):
    for i in range(mb_agent.ensemble_size):
        obs, acs, times = jnp.empty(shape=(batch_size, 201, 3)), jnp.empty(shape=(batch_size, 201, 1)), jnp.empty(shape=(batch_size, 201))
        for m in range(batch_size):
            traj = replay_buffer.sample_rollout()
            obs.at[m].set(traj["observations"])
            acs.at[m].set(traj["actions"])
            times.at[m].set(jnp.cumsum(traj["dts"]))
        loss = mb_agent.batched_update(i=i, obs=obs, acs=acs, times=times, discount=0.9)
# well, on cpu and gpu it all takes around 16 seconds per iteration

100%|██████████| 2/2 [00:31<00:00, 16.00s/it]


In [31]:
acs_key, key = jax.random.split(key)
ob = env.reset()
mb_agent.get_action(obs=ob, key=acs_key)

Array([-0.2797025], dtype=float32)